# Predicción del Engagement en Puntos de Interés Turísticos

## 0. Introducción

Este proyecto tiene como objetivo desarrollar un modelo de **Deep Learning multimodal** capaz de predecir si un punto de interés turístico (POI) generará un nivel **alto o bajo de engagement** entre los usuarios. El término *engagement* se refiere al nivel de interacción que los usuarios tienen con los POIs dentro de una plataforma digital, expresado mediante métricas como visitas, likes, bookmarks o dislikes.

Los datos utilizados en este estudio provienen de la plataforma Artgonuts, garantizando así su relevancia y aplicabilidad en contextos reales
del sector turístico. Las imágenes empleadas han sido específicamente procesadas para los fines de esta práctica, siendo sus versiones originales en alta
resolución procedentes de diversas fuentes, incluyendo el portal de datos
abiertos de la Comunidad de Madrid.

El modelo combinará información de **dos fuentes complementarias**:

- **Características visuales** extraídas de imágenes mediante redes neuronales convolucionales (CNN).
- **Metadatos estructurados** asociados a cada POI (como ubicación, categoría, etiquetas, etc.), procesados mediante redes densas (DNN).

La combinación de ambas fuentes de información en un único modelo permite capturar tanto el impacto visual como el contexto informativo, ofreciendo una capacidad predictiva superior a la de modelos unimodales.

## 1. Descripción del Dataset

El dataset proporcionado contiene información detallada sobre distintos puntos de interés turísticos, y está compuesto por dos partes principales:

- **Imágenes**: Cada POI cuenta con una imagen principal representativa, cuya ruta está especificada en la columna `main_image_path`.

- **Metadatos**: Datos estructurados asociados a cada POI, entre los que se incluyen:
  - Identificador único (`id`)
  - Nombre (`name`)
  - Descripción (`shortDescription`)
  - Ubicación geográfica (`locationLat`, `locationLon`, `distrito`, `barrio`)
  - Categorías (`categories`)
  - Etiquetas (`tags`)
  - Nivel de popularidad (`tier`)
  - Puntos de experiencia (`xps`)
  - Métricas de interacción: `visits`, `likes`, `dislikes`, `bookmarks`

El dataset se encuentra en formato CSV y está vinculado con una carpeta de imágenes.
En este proyecto, trabajaremos con una **variable objetivo construida a partir de las métricas de interacción**, para definir el nivel de engagement de forma binaria (alto vs bajo), y construiremos un modelo que integre imágenes y metadatos para predecir esta variable.


## 2. Estructura general del notebook

El desarrollo del proyecto se organiza en las siguientes etapas, siguiendo buenas prácticas de aprendizaje automático y deep learning:

3. **Carga inicial de datos**  
   Lectura del dataset de metadatos, inspección rápida y eliminación de columnas irrelevantes.

4. **Definición de la variable objetivo (`engagement_target`)**  
   Creación de una métrica sintética de engagement a partir de las interacciones (`visits`, `likes`, `dislikes`, `bookmarks`) y su transformación en una etiqueta binaria: engagement alto o bajo.

5. **División en conjuntos de entrenamiento, validación y prueba**  
   Separación estratificada del dataset en tres subconjuntos. A partir de este punto, todas las decisiones de análisis, normalización y codificación se realizarán exclusivamente sobre el conjunto de entrenamiento para evitar *data leakage*.

6. **Análisis exploratorio y preprocesamiento**  
   - Análisis de valores nulos y distribución de variables.  
   - Imputación, normalización y codificación de variables categóricas.  
   - Preparación final de los datos estructurados para alimentar la red neuronal.

7. **Preparación de imágenes**  
   Carga, redimensionamiento y normalización de imágenes asociadas a cada POI. Se define un `Dataset` que incluye metadatos e imágenes.

8. **Construcción del modelo multimodal**  
   Implementación de una arquitectura que combine:
   - Una red convolucional (CNN) para extraer features visuales.
   - Una red densa (DNN) para procesar metadatos.
   - Una capa de fusión que concatena ambas salidas y alimenta al clasificador final.

9. **Entrenamiento del modelo**  
   Entrenamiento con validación, utilizando Binary Cross Entropy como función de pérdida y estrategias de regularización como Dropout y Early Stopping.

10. **Evaluación del modelo**  
   Evaluación del rendimiento en el conjunto de validación utilizando métricas como Accuracy, Precision, Recall, F1-score y matriz de confusión.

11. **Conclusiones**  
   Reflexión sobre los resultados obtenidos, principales aprendizajes y posibles líneas de mejora futura.

12. **Reproducibilidad**  
    Se documenta el entorno utilizado (`requirements.txt`) y se asegura la fijación de semillas aleatorias para permitir la replicabilidad del experimento.


## 3. Carga inicial de datos

En esta sección se realiza la carga inicial del dataset de metadatos turísticos proporcionado en formato CSV. El objetivo es verificar su estructura general, revisar las columnas disponibles y eliminar aquellas que no aportan valor predictivo para el modelo.

> ⚠️ Nota importante: **aún no se realiza ninguna transformación estadística sobre los datos**, ni análisis exploratorio detallado, ni imputación. Todas esas acciones se llevarán a cabo exclusivamente sobre el conjunto de entrenamiento una vez realizada la partición del dataset, para evitar *data leakage*.

### Acciones realizadas:
- Lectura del archivo CSV principal (`poi_dataset.csv`).
- Revisión de número de filas y columnas.
- Inspección de los nombres, tipos de datos y primeras filas.
- Eliminación de columnas obvias que no serán utilizadas como input ni target.


In [ ]:
# 3. Carga inicial de datos

import pandas as pd

# Cargar el dataset
df = pd.read_csv("poi_dataset.csv")

# Mostrar dimensiones del dataframe
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

# Ver columnas disponibles
print("\nColumnas disponibles:")
print(df.columns.tolist())

# Vista rápida de los primeros registros
df.head()
